# Data Acquisition Example: Testing WLAN Client Receiver System Noise

Test data for WLAN is implemented with the the iperf support in [ssmdevices](https://gitlab.nist.gov/ssm/ssmdevices). It produces many more columns of data than we need, so we just use the two defined in `ipc_columns`: throughput and a timestamp.

The results of the tests and the corresponding test conditions are stored in a [flat database](https://en.wikipedia.org/wiki/Flat_file_database) stored in SQLite format. It is implemented as a typical intended use case of `labbench.RelationalDataInSQLite`. The test conditions are implemented as states in the attenuators, `iperf`, and `wlan`, so all we need to log these results into the database is to add the `db.on_set` call. Any states that are changed after that function call (for example, with the for loops in the acquisition code) become columns in the database; these values are automatically kept up to date and written to the database on calls to `db.write`.

In [2]:
# This should be run in python 3.6.x from the computer connected to the AP side

%pylab inline
import time,os
import ssmdevices as ssm
import labbench as lb
import pandas as pd

def meshpoints (x1, x2, *args):
    ''' For input of N-dimensions of grid point vectors (x1, x2, ..., xN),
        where each (x1, x2, ... xN) has dimension (M1, M2, ..., MN),
        returns an array with shape (M1*M2*...*MN,N) that consists of the
        unique points in the multi-dimensional grid with sampled at
        the vector points (x1, x2, ..., xN) in each axis. This is also known
        as the the cartesian product of (x1 ... xN).
    '''
    ret = np.array(np.meshgrid(*((x1,x2)+args)))
    return ret.reshape([ret.shape[0],np.prod(ret.shape[1:])]).T

setup_time = 5 # seconds
acquisition_time = 30 #  seconds

# iperf_columns       = ['iperf_bits_per_second','iperf_timestamp']
sweep_points       = meshpoints(list(range(55,76,1)),[110]+list(range(0,21,1)))

lb.show_messages('info')

class Testbed(lb.Testbed):
    def make(self):
        ''' lb.Testbed calls this automatically when we call Testbed(config)
        '''
        self.c = ssm.instruments.MiniCircuitsRCDAT('11604210008')
        self.e0 = ssm.instruments.MiniCircuitsRCDAT('11604210014')
        self.iperf_client = ssm.software.IPerf('10.0.0.2', bind='10.0.0.3', port=5010, interval=0.1, buffer_size=64, tcp_window_size=4096)
        self.iperf_server = ssm.software.IPerf(bind='10.0.0.2', port=5010, buffer_size=64, tcp_window_size=4096)
        self.wlan = ssm.software.WLANStatus('Wi-Fi', ssid='EnGenius1')
        self.db = lb.StatesToSQLite(time.strftime("%Y-%m-%d-%H%M%S with shuffle and absorber"))
        self.dc_power = ssm.instruments.RigolDP800Series('USB0::0x1AB1::0x0E11::DP8C180200079::INSTR')

        # Log all state changes in lte_laa and each client in iperf_clients
        self.db.observe_states([self.c,self.e0,self.iperf_server,self.iperf_client,self.wlan])
        
    def acquire(self, duration):
        ''' acquire for `duration` seconds and return a dictionary of
            client and server data frames
        '''
        # Cycle through port in iperf server & client to avoid some strange
        # errors involving stale ports in windows
        p = self.iperf_server.settings.port
        self.iperf_client.settings.port = self.iperf_server.settings.port = p+1            
        
        self.iperf_server.start()
        self.iperf_client.start()

        lb.logger.info('acquiring for {}s'.format(duration))
        lb.sleep(duration)

        client = self.iperf_client.fetch()
        server = self.iperf_server.fetch()

        lb.logger.debug('  iperf_client and server returned {} and {} rows'\
                       .format(len(client),len(server)))

        self.iperf_server.kill()
        self.iperf_client.kill()
        
        return {'iperf_client': client,
                'iperf_server': server}

    def startup(self):
        lb.panel(self, ncols=6) # Heads-up display for the notebook
        
        # Noise source sanity checks
        if not self.dc_power.state.enable1:
            raise ValueError('noise source power supply is off. check voltage setting, connect, and turn on')
        if self.dc_power.state.voltage1 < 10:
            raise ValueError('noise source voltage supply setting is low, {}V'\
                             .format(self.dc_power.state.voltage1))
        if self.dc_power.state.current1 < 0.07:
            raise ValueError('noise source current draw is low, {}A. check connections'\
                             .format(self.dc_power.state.current1))
        
        lb.logger.info('setup start')
        self.t0 = time.time()
        
        self.wlan.interface_disconnect()
        
        self.c.state.attenuation = 50
        self.e0.state.attenuation = 110
        
        # Throwaway acquisition to get into a favorable state
        self.acquire(setup_time)    
        lb.logger.info('setup done')

    def cleanup(self):
        # End any stale iperf processes in the correct order
        self.iperf_server.kill()
        self.iperf_client.kill()
        
        # Leave attenuators in a sensible state
        self.c.state.attenuation = 50
        self.e0.state.attenuation = 110
        
        # Load the sqlite database and save a copy into a csv
        df = lb.read(os.path.join(self.db.path, 'master.db'))
        df.to_csv(os.path.join(self.db.path,'master.csv'))
        
        # Flatten the iperf result
        lb.logger.info('expanding iperf result into summary output')
        df = lb.read_relational(os.path.join(self.db.path, 'master.db'),
                                'iperf_client', prepend_column_name=False,
                                target_cols=['iperf_destination_address','iperf_destination_port', 'iperf_bits_per_second', 'iperf_timestamp'],)
        df.to_csv(os.path.join(self.db.path, 'summary.csv'))
        lb.to_feather(df, os.path.join(self.db.path, 'summary.feather'))
        
        lb.logger.info('finished with testbed after {}s'\
                       .format(time.time()-self.t0))

for i in range(2):
    states = {}

    np.random.shuffle(sweep_points)
    
    # This `with' block ensures sockets and iperf subprocesses are
    # closed correctly when the script ends, even if exceptions are raised
    with Testbed() as testbed:
        for testbed.c.state.attenuation, testbed.e0.state.attenuation\
            in lb.log_progress(sweep_points):

            try:
                row = {}                
                testbed.wlan.interface_reconnect(5)     # Fresh connection
                row = testbed.acquire(acquisition_time)
            except TimeoutError as e:
                # Blank data
                lb.logger.debug(str(e))
                lb.logger.debug('No connection to AP - skipping data acquisition')
            finally:
                # Update all WLAN states before append 
                testbed.wlan.refresh()
                testbed.db.append(row)
                testbed.db.write()
        print('finished iteration ',i)

Populating the interactive namespace from numpy and matplotlib


2018-11-02 16:17:14.659 - INFO - Connected in 0.68s


C:\Users\dkuester\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


C:\Users\dkuester\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


finished iteration  0


finished iteration  1
